In [ ]:
%run generator/statementGenerator.ipynb
%run generator/IDsPlayerTimeGenerator.ipynb
%run generator/TypeChecker.ipynb

**Select number of traces, players and IDs** to generate

In [ ]:
nbtracesCompletable = widgets.IntSlider(description="Traces", min=0, value=1, max=10000, continuous_update=False)
nbtracesCompletabletext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbtracesCompletable, 'value'), (nbtracesCompletabletext, 'value'))
tracesCompletable=VBox([nbtracesCompletable,nbtracesCompletabletext])

nbplayersCompletable = widgets.IntSlider(description="Players", min=0, value=1, continuous_update=False)
nbplayersCompletabletext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbplayersCompletable, 'value'), (nbplayersCompletabletext, 'value'))
widgets.link((nbtracesCompletable, 'value'), (nbplayersCompletable, 'max'))
playersCompletable=VBox([nbplayersCompletable,nbplayersCompletabletext])

nbIDsCompletable = widgets.IntSlider(description="IDs", min=0, value=1, continuous_update=False)
nbIDsCompletabletext = widgets.IntText(description="", continuous_update=False)
widgets.link((nbIDsCompletable, 'value'), (nbIDsCompletabletext, 'value'))
widgets.link((nbtracesCompletable, 'value'),(nbIDsCompletable, 'max'))
IDsCompletable=VBox([nbIDsCompletable,nbIDsCompletabletext])

vbox_nb_traces_ID_player_completable=VBox([tracesCompletable,playersCompletable, IDsCompletable])

**Completables**

In [ ]:
types_completables=["serious-game", "game", "session", "level", "quest", "stage", "combat", "storynode", "race", "completable"]
checkboxes_types_completables=GeneratorMultiCheckboxWidget("Completables types", types_completables)

verbs_completables=["initialized", "progressed", "completed"]
checkboxes_verbs_completables=GeneratorMultiCheckboxWidget("Completables verbs", verbs_completables)

template_result_progress={
    "extensions": {
		"https://w3id.org/xapi/seriousgames/extensions/progress": None,
		"progress": None
	}
}

template_result_completed_score={
	"extensions": {
		"success": None,
		"score":None
	}
}

def generate_completable(change=None):
    start_time=datetime.now()
    if change!=None:
        with output:
            print("Starting the data generation")
    verbs=checkboxes_verbs_completables.get_selected_options()
    types=checkboxes_types_completables.get_selected_options()
    ids=generate_IDs(nbIDsCompletable.value)
    times=generate_times_datas(nbtracesCompletable.value)
    player=generate_players(nbplayersCompletable.value)
    statements=[]
    checkCompletable={}
    for i in range(nbtracesCompletable.value):
        randomID=random.choice(ids)
        randomPlayer=random.choice(player)
        randomType = checkType(checkCompletable,randomID,randomPlayer,types)
        if checkCompletable[randomType][randomID][randomPlayer] in [None, False]:
            checkCompletable[randomType][randomID][randomPlayer]=True
            randomVerb="initialized"
        else:
            randomVerb=random.choice(["progressed", "completed"])
            if randomVerb=="completed":
                checkCompletable[randomType][randomID][randomPlayer]=False
        time=times[i]
        statement=generator(randomPlayer, randomVerb, randomType,randomID, time)
        if randomVerb=="completed":
            result=copy.deepcopy(template_result_completed_score)
            result["extensions"]["score"]=random.random()*100
            result["extensions"]["success"]=random.choice([True,False])
            statement["result"]=result
        elif randomVerb=="progressed":
            result=copy.deepcopy(template_result_progress)
            progress=random.random()
            result["extensions"]["https://w3id.org/xapi/seriousgames/extensions/progress"]=progress
            result["extensions"]["progress"]=progress
            statement["result"]=result
        statements.append(statement)
    if change==None:
        return statements
    else:
        save_generated_data(statements, verbs, types)
        end_time=datetime.now()
        delta=end_time-start_time
        with output:
            print("End the data generation")
            print("Generated in "+str(delta))
            clear_output(wait=True)

buttonCompletable=widgets.Button(description='Generate Completable Data', layout = widgets.Layout( width='auto'))
buttonCompletable.on_click(generate_completable)

global tabCompletable
tabCompletable=VBox([
    HBox([
        checkboxes_verbs_completables.multi_select,
        checkboxes_types_completables.multi_select,
        vbox_nb_traces_ID_player_completable
    ]),
    buttonCompletable
])